# Actors' operation

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../..")

## Manipulate an individual `Actor`

In [2]:

from tests.create_tested_instances import simple_main_model
from abses.actor import Actor

# create a testing model
model = simple_main_model('actor_example')

# actor creation should be bounded with a model.
actor = Actor(model)

# actor has a class-property, -its breed, class's name.
actor.breed  # same as `actor.__class__.__name__`

'Actor'

In [3]:
# In an initial condition, this actor is not on the earth...
actor.on_earth

False

In [4]:
# however, we can let it settle down on a specific position of the earth.
pos = (4, 4)
settled = actor.settle_down(position=pos)

actor.pos  # position of the actor now: (4, 4)
actor.on_earth  # True

(4, 4)

True

*Note that `Actor`s created in this way is not accessible from the model, unless we add it into the model.*

In [5]:
actor in model.agents  # False

# `register` must be assigned with True, because it's a new breed for this model.
model.agents.add(actor, register=True)

actor in model.agents  # True

False

True

Actor is an `BaseObj` of the model, so it inherits [all of its properties](#TODO).


In [6]:
# default, it's name is as same as its breed in lower case
actor.name  # 'Actor'

# if assigned name when creating, they will be different.
actor2 = Actor(model, name='actor2')
actor2.breed
actor2.name

'actor'

'Actor'

'actor2'

## Manipulate a group of `Actors`
`ABSESpy` provides two different container of actors:
- `AgentsContainer`: A unique `dictionary`-like class where all agents of the model are saving.
- `ActorsList`: A `list`-like class where referring some actors temporally.

### AgentsContainer



When the model is created, there is a unique `AgentsContainer` obj bounding. It saves agents by their breeds.

In [7]:
type(model.agents)
model.agents  # only one actor is added now.

model.agents.breeds  # breeds

abses.container.AgentsContainer

<AgentsContainer: ActorsList: (1)Actor>

('Actor',)

Container makes creating actors much easier:

In [8]:
# define a new breed of actor
class Seller(Actor):
    pass


class User(Actor):
    pass


another_actor = model.agents.create(Actor)
seller = model.agents.create(Seller)  # default creating one actor.
users = model.agents.create(User, 5) # creating 5 actors

model.agents


<AgentsContainer: ActorsList: (2)Actor; (1)Seller; (5)User>

While `AgentsContainer` mainly provides a way to store, most of manipulations for actors are implemented in another data type: `ActorsList`. `AgentsContainer` also has many ways to convert existing agents to this data type:

In [9]:
# generate all existing agents to an `ActorsList`.
model.agents.to_list()

<ActorsList: (2)Actor; (1)Seller; (5)User>

In [10]:
# select specific breeds.
model.agents.to_list(breeds='Seller')
model.agents.to_list(breeds=['Seller', 'User'])

<ActorsList: (1)Seller>

<ActorsList: (1)Seller; (5)User>

In [11]:
# also accessible through attributes.
model.agents.User

<ActorsList: (5)User>

### `ActorsList`

`ActorsList` is a collection of `Actors`, facilitate to manipulate them at a batch.

In [12]:
lst = model.agents.to_list()
actor = lst[0]  # indexing -> an `Actor`
five_actors = lst[:5]  # slice -> another `ActorsList` object

five_actors  # (2)Actor; (1)Seller; (2)User
# five_actors contains this actor
actor in five_actors

<ActorsList: (2)Actor; (1)Seller; (2)User>

True

Convert the list to a dictionary: `{breed: ActorsList}`

In [13]:
# sort up mixed actors.
five_actors.to_dict()

{'Actor': <ActorsList: (2)Actor>,
 'Seller': <ActorsList: (1)Seller>,
 'User': <ActorsList: (2)User>}

Select by conditions.

In [14]:
five_actors.select(selection='User')  # same as .to_dict()['User']

# select the first and the fourth actors
five_actors.select(selection=[True, False, False, True, False])

<ActorsList: (2)User>

<ActorsList: (1)Actor; (1)User>

Actors' attributes are accessible as `np.ndarray`.

In [15]:
# get each Actor(object)'s id.
five_actors.array('id')

# alternative way to get this attribute.
five_actors.id

array([ 4,  6,  7,  8, 12])

array([ 4,  6,  7,  8, 12])

You can access all actors in this list with `id >= 7` by:

In [16]:
better_guys = five_actors.better(metric='id', than=7)
better_guys.id

array([ 8, 12])

Split the list in the second, third places: `[1, 2, | 3, | 4, 5]`

In [17]:
five_actors.split([2, 3])

[array([Actor (Obj 4), Actor (Obj 6)], dtype=object),
 array([Seller (Obj 7)], dtype=object),
 array([User (Obj 8), User (Obj 12)], dtype=object)]